## Extract Information from Nested Json file into CSV File

In [2]:
import json
import objectpath
import pandas as pd

In [2]:
with open('json/audit.json',encoding="utf8") as f:
    data = json.load(f)

In [3]:
data

{'@xmlns': 'http://www.cdisc.org/ns/odm/v1.3',
 '@xmlns:OpenClinica': 'http://www.openclinica.org/ns/odm_ext_v130/v3.1',
 '@xmlns:OpenClinicaRules': 'http://www.openclinica.org/ns/rules/v3.1',
 '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
 '@FileOID': 'Study-MetaD20200315170459+0000',
 '@Description': 'Study Metadata',
 '@CreationDateTime': '2020-03-15T17:04:59+00:00',
 '@FileType': 'Snapshot',
 '@ODMVersion': '1.3',
 '@xsi:schemaLocation': 'http://www.cdisc.org/ns/odm/v1.3 OpenClinica-ODM1-3-0-OC3-0.xsd',
 'ClinicalData': {'@StudyOID': 'S_ABTX5100(TEST)',
  '@MetaDataVersionOID': None,
  'SubjectData': {'@xmlns:OpenClinica': 'http://www.openclinica.org/ns/odm_ext_v130/v3.1',
   '@SubjectKey': 'SS_Site001',
   '@OpenClinica:StudySubjectID': 'Site001',
   '@OpenClinica:Status': 'Available',
   'StudyEventData': [{'@StudyEventOID': 'SE_DEV_VISIT',
     '@OpenClinica:EventName': 'Screening',
     '@OpenClinica:StartDate': '16-Jan-2020',
     '@OpenClinica:Status': 'data entr

In [4]:
jsonnn_tree = objectpath.Tree(data['ClinicalData'])

In [5]:
result_list = list(jsonnn_tree.execute('$..ItemData'))

In [6]:
result_list

[{'@ItemOID': 'I_CLINI_ALB',
  '@OpenClinica:ItemName': 'ALB',
  '@Value': '5',
  'OpenClinica:AuditLogs': {'@EntityID': 'I_CLINI_ALB',
   'OpenClinica:AuditLog': {'@ID': 'AL_15319',
    '@UserID': 'USR_43',
    '@UserName': 'ismith',
    '@Name': 'Indar Smith',
    '@DateTimeStamp': '07-Feb-2020 14:47:03',
    '@AuditType': 'Item data value updated',
    '@ReasonForChange': 'initial value',
    '@NewValue': '5',
    '@ValueType': 'ALB'}}},
 {'@ItemOID': 'I_CLINI_ALP',
  '@OpenClinica:ItemName': 'ALP',
  '@Value': '35',
  'OpenClinica:AuditLogs': {'@EntityID': 'I_CLINI_ALP',
   'OpenClinica:AuditLog': {'@ID': 'AL_15321',
    '@UserID': 'USR_43',
    '@UserName': 'ismith',
    '@Name': 'Indar Smith',
    '@DateTimeStamp': '07-Feb-2020 14:47:06',
    '@AuditType': 'Item data value updated',
    '@ReasonForChange': 'initial value',
    '@NewValue': '35',
    '@ValueType': 'ALP'}}},
 {'@ItemOID': 'I_CLINI_ALT',
  '@OpenClinica:ItemName': 'ALT',
  '@Value': '620',
  'OpenClinica:AuditLogs':

In [7]:
fieldname=[]
newvalue=[]
oldvalue=[]
timestamp=[]
user=[]
atype=[]

for item in result_list:
    itemname=item['@OpenClinica:ItemName']
    auditlog=item['OpenClinica:AuditLogs']['OpenClinica:AuditLog']
    if isinstance(auditlog, dict):
        old=0 # check if there is @OldValue key. Not every @NewValue has a @OldValue
        new=0 # check if there is @NewValue key. Not every @OldValue has a @NewValue
        for key, value in auditlog.items():
            if key== '@NewValue':
                new=1
                newvalue.append(value)
            if key== '@OldValue':
                old=1
                oldvalue.append(value)          
            if key== "@DateTimeStamp":
                DateTimeStamp=value
            if key== "@UserName":
                UserName=value
            if key== "@AuditType":
                AuditTypee=value
            if key== "@ReasonForChange":
                ReasonForChange=value
        fieldname.append(itemname)
        timestamp.append(DateTimeStamp)
        user.append(UserName)
        atype.append(AuditTypee)
        if old==0 and new==1:
            oldvalue.append('') # if no @oldValue, still add a blank value
        if new==0 and old==1:
            newvalue.append('') # if no @NewValue, still add a blank value
    elif isinstance(auditlog,list):
        for dd in auditlog:
            old=0 # check if there is @OldValue key. Not every @NewValue has a @OldValue
            new=0 # check if there is @NewValue key. Not every @OldValue has a @NewValue            
            for key, value in dd.items():
                if key== '@NewValue':
                    new=1
                    newvalue.append(value)
                if key== '@OldValue':
                    old=1
                    oldvalue.append(value)
                if key== "@DateTimeStamp":
                    DateTimeStamp=value
                if key== "@UserName":
                    UserName=value
                if key== "@AuditType":
                    AuditTypee=value
                if key== "@ReasonForChange":
                    ReasonForChange=value
            fieldname.append(itemname)
            timestamp.append(DateTimeStamp)
            user.append(UserName)
            atype.append(AuditTypee)
        
            if old==0 and new==1:
                oldvalue.append('') # if no @oldValue, still add a blank value
            if new==0 and old==1:
                newvalue.append('') # if no @NewValue, still add a blank value
#len(fieldname),len(timestamp),len(newvalue),len(oldvalue)
auditdf = {'fieldname': fieldname, 'newvalue': newvalue, 'oldvalue': oldvalue,'User':user,'timestamp':timestamp,'type':atype}  
df = pd.DataFrame(auditdf)   
df.to_csv('audit_log.csv', index=False)

In [9]:
df.tail(20)

,fieldname,newvalue,oldvalue,User,timestamp,type
284,TRTFREQ,BID,,ismith,16-Jan-2020 19:39:59,Item data value updated
285,TRTFREQ,QD,BID,ismith,16-Jan-2020 19:41:02,Item data value updated
286,TRTIND,neutropenia,,ismith,16-Jan-2020 19:41:09,Item data value updated
287,TRTROUTE,SQ,,ismith,16-Jan-2020 19:40:24,Item data value updated
288,TRTSTDT,01-Jan-2020,,ismith,16-Jan-2020 19:41:10,Item data value updated
289,TRTUNIT,gtt,,ismith,16-Jan-2020 19:39:57,Item data value updated
290,TRTUNIT,OTH,gtt,ismith,16-Jan-2020 19:40:40,Item data value updated
291,TRTUNIT,ug,OTH,ismith,16-Jan-2020 19:40:57,Item data value updated
292,CMDOSFRQ,BID,,umessier,08-Feb-2020 23:28:31,Item data value updated
293,CMDOSTXT,500,,umessier,08-Feb-2020 23:28:07,Item data value updated


In [3]:
df = pd.DataFrame({'type': []}) 
df

,type
